In [32]:
import tkinter as tk
from tkinter import ttk
import math
import random
import numpy as np

In [33]:
class Ball:
    def __init__(self, canvas, x, y, xspeed = 0, yspeed = 0, xaccel = 0, yaccel = 0, size = 25, colour="white"):
        self.canvas = canvas
        self.col = "white"
        self.x = x
        self.y = y
        self.xspeed = xspeed
        self.yspeed = yspeed
        self.xaccel = xaccel
        self.yaccel = yaccel
        self.size = size
        self.colour = colour
        self.prot = 15
        self.index = len(balls)
   #     self.checkCollisions()
        self.flagged = np.zeros(2000)

    def move(self):
        self.canvas.create_oval(
            self.x - self.size / 2, self.y - self.size / 2,
            self.x + self.size / 2, self.y + self.size / 2,
            fill=self.colour
        )
        self.xspeed += self.xaccel
        self.yspeed += self.yaccel
        self.x += self.xspeed
        self.y += self.yspeed
        
        if self.x > self.canvas.winfo_width() - self.size / 2:
            self.xspeed = -abs(self.xspeed) #assuming collisions with walls is totally elastic
            self.x=self.canvas.winfo_width()-self.size/2
            playSynth()
        elif self.x < self.size/2:
            self.xspeed = abs(self.xspeed)
            self.x=self.size/2
        if self.y > self.canvas.winfo_height() - self.size / 2:
            self.yspeed = -abs(self.yspeed)
            self.y=self.canvas.winfo_height()-self.size/2
            playSynth()
        elif self.y < self.size/2:
            self.yspeed = abs(self.yspeed)
            self.y=self.size/2

    def speed(self):
        return np.sqrt(self.xspeed**2+self.yspeed**2)

In [5]:
def playSynth():
    # Add your playSynth logic here
    pass

def distance(x1, y1, x2, y2):
    dX = abs(x1 - x2)
    dY = abs(y1 - y2)
    return (dX ** 2 + dY ** 2) ** 0.5

def setOrigCoords(event):
    global origX, origY
    origX, origY = event.x, event.y

def addBall(ballX, ballY, ballVelX, ballVelY, ballAccelX=0, ballAccelY=0):
    global l
    balls.append(Ball(canvas, ballX, ballY, ballVelX, ballVelY, ballAccelX, ballAccelY, colour = random.choice(colours)))
    l += 1
    
def addBallEvent(event):
    newX, newY = event.x, event.y
    addBall(newX, newY, (newX-origX/10, (newY-origY)/10))
    
def dotProduct(x1,y1,x2,y2):
    return x1*x2 + y1*y2

def VSub(x1, y1, x2, y2):
    return x1-x2, y1-y2
    

def calculateNewVel(x1, y1, m1, vx1, vy1, x2, y2, m2, vx2, vy2, l=0):
    scal = (2*m2/(m1+m2))*(dotProduct(vx1-vx2, vy1-vy2, x1-x2, y1-y2)/dotProduct(x1-x2, y1-y2, x1-x2, y1-y2))
    posDifX, posDifY = VSub(x1,y1,x2,y2)
    #writeLog(f"{scal}",2)
    newVelx1, newVely1 = VSub(vx1, vy1, scal*posDifX, scal*posDifY)
    if l:
        return newVelx1, newVely1
    else:
        newVelx2, newVely2 = calculateNewVel(x2, y2, m2, vx2, vy2, x1, y1, m1, vx1, vy1, l=1)
    return newVelx1, newVely1, newVelx2, newVely2

def pushBackAlongAxis(x,y,xspeed,yspeed,z,a,b,aspeed,bspeed,c):
    overlap = z+c-distance(x,y,a,b) #the pink
    theta = math.atan((x-a)/(y-b))
    movex = math.cos(theta)*overlap
    movey = math.sin(theta)*overlap
    xprop = xspeed/aspeed
    yprop = yspeed/bspeed
    newx = movex*xprop
    newy = movey*yprop
    return newx,newy
    


def detectCollisions(balls):
    checked = np.zeros(len(balls)) #initialise list of checked to avoid unecessary checks
    for i in range(len(balls)): #for every ball
        if not checked[i]: #if it has not been checked
            for j in range(len(balls)): #check against every other ball
                if i!=j and not checked[j]: #which had not been checked yet
                    if distance(balls[i].x, balls[i].y, balls[j].x, balls[j].y) <= (balls[i].size+balls[j].size)/2: #if they are touching
                        writeLog(f"{j} touching {i}\n", 1)
                        bpushBackAlongAxis(balls[i].x,balls[i].y,balls[i].xspeed,balls[i].yspeed,balls[i].size,balls[j].x,balls[j].y,balls[j].xspeed,balls[j].yspeed,balls[j].size)
                        pushBackAlongAxis(balls[j].x,balls[j].y,balls[j].xspeed,balls[j].yspeed,balls[j].size,balls[i].x,balls[i].y,balls[i].xspeed,balls[i].yspeed,balls[i].size)
                        if not balls[i].flagged[j]: #if previous frame they also were not touching
                            balls[i].xspeed, balls[i].yspeed, balls[j].xspeed, balls[j].yspeed = calculateNewVel( #change velocity 
                                balls[i].x,balls[i].y,balls[i].size**2,balls[i].xspeed,balls[i].yspeed,
                                balls[j].x,balls[j].y,balls[j].size**2,balls[j].xspeed,balls[j].yspeed)
                            checked[j]=1
                            balls[i].flagged[j]=1 #add to list of recenrlt touched
                            balls[j].flagged[i]=1
                            writeLog(f"valid collision!\n",1)
                    else:
                        if balls[i].flagged[j]:
                            balls[i].flagged[j]=0
                            balls[j].flagged[i]=0
            checked[i]=1
            

In [31]:
G = 0
colours = ["white", "maroon", "teal", "lime", "chocolate", "gold", "aqua"]
delay = 30

writeFile = open("log/log.txt", "wt")
loggingLevel = 1 #0 = nothing, 1 = errors only, 2 = all

def writeLog(message, level=0, file = writeFile):
    if level <= loggingLevel:
        file.write(message)

writeLog("starting simulation\n", 2, writeFile)

root = tk.Tk()
root.title("Ball Bouncing Simulator")

canvas = tk.Canvas(root, width=500, height=400, bg="black")
canvas.pack()

frame = tk.Frame(root)
frame.pack()

#frame.columnconfigure(0,weight=1)
#frame.columnconfigure(1,weight=2)
"""
framerate = ttk.Frame(root, from_=10, to=50, orient="horizontal", variable=delay)
framerate.pack()
"""

ballx = tk.Entry(frame,width=10)
ballx.insert(0, "10")
ballx.grid(column=1, row=0)
#ballx.pack(padx=1,pady=1)

ballxlabel = tk.Label(frame, text="X")
#ballxlabel.pack(side=tk.LEFT)
ballxlabel.grid(column=0, row=0)


bally = tk.Entry(frame, width=10)
bally.insert(0,"50")
#bally.pack(padx=1,pady=1)
bally.grid(column=1, row=1)


ballylabel = tk.Label(frame, text="Y")
#ballylabel.pack(side=tk.LEFT)
ballylabel.grid(column=0, row=1)


ballvelx = tk.Entry(frame, width=10)
ballvelx.insert(0,"5")
#ballvelx.pack(padx=1,pady=1)
ballvelx.grid(column=1, row=2)

ballxvellabel = tk.Label(frame, text="X velocity")
#ballxvellabel.pack(side=tk.LEFT)
ballxvellabel.grid(column=0, row=2)


ballvely = tk.Entry(frame, width=10)
ballvely.insert(0,"0")
#ballvely.pack(padx=1,pady=1)
ballvely.grid(column=1, row=3)

ballyvellabel = tk.Label(frame, text="Y velocity")
#ballyvellabel.pack(side=tk.LEFT)
ballyvellabel.grid(column=0, row=3)


def addBallWithParams():
    addBall(int(ballx.get()), int(bally.get()), int(ballvelx.get()), int(ballvely.get()), ballAccelY=G)
    
AddBall = tk.Button(frame, text="Submit", command = addBallWithParams)
#AddBall.pack(padx=5, pady=5)
AddBall.grid(column=1, row=4, padx=5, pady=5)

balls = []
origX, origY = 0, 0
l = 0

#canvas.bind("<Button-1>", setOrigCoords)
#canvas.bind("<ButtonRelease-1>", addBall)

def update():
    canvas.delete("all")
    for ball in balls:
        ball.move()
    detectCollisions(balls)
    root.after(delay, update)
    
update()

root.mainloop()

writeFile.close()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Boguslavskaya Masha\anaconda3\envs\tkinter\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Boguslavskaya Masha\anaconda3\envs\tkinter\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\Boguslavskaya Masha\AppData\Local\Temp\ipykernel_28064\2790916355.py", line 89, in update
    detectCollisions(balls)
  File "C:\Users\Boguslavskaya Masha\AppData\Local\Temp\ipykernel_28064\3688160887.py", line 51, in detectCollisions
    balls[i].xspeed, balls[i].yspeed, balls[j].xspeed, balls[j].yspeed = calculateNewVel( #change velocity
                                                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Boguslavskaya Masha\AppData\Local\Temp\ipykernel_28064\3688160887.py", line 31, in calculateNewVel
    scal = (2*m2/(m1+m2))*(dotProduct(vx1-vx2, vy1-vy2, x1-x2, y1-